In [1]:
import ast
import copy
import re

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from aocd import get_data, submit

DAY = 24
YEAR = 2022

In [2]:
# use test data
raw_test = """#.######
#>>.<^<#
#.<..<<#
#>v.><>#
#<^v^^>#
######.#"""

# use real data
raw = get_data(day=DAY, year=YEAR)

print(raw_test)

#.######
#>>.<^<#
#.<..<<#
#>v.><>#
#<^v^^>#
######.#


In [3]:
symbols = {"#": -1, ">": 0, "v": 1, "<": 2, "^": 3}
symbols_inv = {v: k for k, v in symbols.items()}


def parse_data(data):
    global symbols

    s, e, wall, bliz = None, None, set(), {}
    data = data.split("\n")
    for rdx, row in enumerate(data):
        for cdx, el in enumerate(row):
            if rdx == 0 and el == ".":
                s = (cdx, rdx)
            elif rdx == len(data) - 1 and el == ".":
                e = (cdx, rdx)
            elif el in symbols:
                if el == "#":
                    wall.add((cdx, rdx))
                else:
                    if (cdx, rdx) not in bliz:
                        bliz[(cdx, rdx)] = set()
                    bliz[(cdx, rdx)].add(symbols[el])

    wall.add((s[0], s[1] - 1))
    wall.add((e[0], e[1] + 1))
    return wall, bliz, s, e, len(data) - 2, len(data[0]) - 2


dummy = parse_data(raw_test)
real = parse_data(raw)

dummy


({(0, 0),
  (0, 1),
  (0, 2),
  (0, 3),
  (0, 4),
  (0, 5),
  (1, -1),
  (1, 5),
  (2, 0),
  (2, 5),
  (3, 0),
  (3, 5),
  (4, 0),
  (4, 5),
  (5, 0),
  (5, 5),
  (6, 0),
  (6, 6),
  (7, 0),
  (7, 1),
  (7, 2),
  (7, 3),
  (7, 4),
  (7, 5)},
 {(1, 1): {0},
  (2, 1): {0},
  (4, 1): {2},
  (5, 1): {3},
  (6, 1): {2},
  (2, 2): {2},
  (5, 2): {2},
  (6, 2): {2},
  (1, 3): {0},
  (2, 3): {1},
  (4, 3): {0},
  (5, 3): {2},
  (6, 3): {0},
  (1, 4): {2},
  (2, 4): {3},
  (3, 4): {1},
  (4, 4): {3},
  (5, 4): {3},
  (6, 4): {0}},
 (1, 0),
 (6, 5),
 4,
 6)

# Part 1

In [13]:
directions = {
    0: [1, 0],
    1: [0, 1],
    2: [-1, 0],
    3: [0, -1],
}

def run_blizzard(bliz):
    global wall
    global nrows
    global ncols
    global directions

    new_bliz = {}
    for current, bls in bliz.items():
        for bl in bls:
            new = tuple([c + d for c, d in zip(current, directions[bl])])
            if new in wall:
                nx, ny = new
                new = tuple([(nx - 1) % ncols + 1, (ny - 1) % nrows + 1])

            if new not in new_bliz:
                new_bliz[new] = set()

            new_bliz[new].add(bl)
    return new_bliz


def get_blizzard_period(bliz):
    global nrows
    global ncols

    c = 0
    while c <= nrows*ncols:
        c += 1
        if c % nrows == 0 and c % ncols == 0:
            break
    
    return c


def get_blizzard_states(bliz):
    period = get_blizzard_period(bliz)

    states = {}
    for state_idx in range(period):
        states[state_idx] = bliz.copy()
        bliz = run_blizzard(bliz)

    return states, period


def get_possible_moves(blizzard_states, current):
    global wall
    global directions
    global period

    moves = set()
    (cx, cy), ct = current
    wait_time = period
    for wt in range(1, period+1):
        if (cx, cy) in blizzard_states[(ct + wt) % period]:
            wait_time = wt + 1
            break

    for wt in range(1, wait_time):
        nbs = {((cx + x, cy + y), ct + wt) for x, y in directions.values() if (cx + x, cy + y) not in wall}
        nbs = {nb for nb in nbs if nb[0] not in blizzard_states[nb[1] % period].keys()}
        moves |= nbs

    return moves


In [21]:
def dfs(visited, start, end, node=None, best=None):
    global wall
    global nrows
    global ncols
    global bliz_states
    global period

    if node is None:
        node = start

    if node[0] == end:
        if best is None or node[1] <= best[1]:
            best = node
            print(best)
        return best

    # test
    # if node[1] > nrows*nco:
        # return best

    if (node[0], node[1] % period) not in visited:
        visited.add((node[0], node[1] % period))

        next_moves = get_possible_moves(bliz_states, node) - visited
        next_moves = sorted(
            next_moves, key=lambda c: abs(c[0][0] - end[0]) + abs(c[0][1] - end[1]) + abs(c[1] - start[1])
        )
        for move in next_moves:
            best = dfs(visited, start, end, move, best)

    return best


In [22]:
data = copy.deepcopy(real)
wall, bliz, start, end, nrows, ncols = data
bliz_states, period = get_blizzard_states(bliz)

visited = set()
result = dfs(visited, (start, 0), end)[1]
result


((120, 26), 555)


RecursionError: maximum recursion depth exceeded while calling a Python object

In [20]:
visited

{((113, 23), 446),
 ((119, 25), 58),
 ((120, 23), 104),
 ((120, 25), 417),
 ((114, 18), 172),
 ((116, 22), 110),
 ((114, 23), 242),
 ((116, 25), 584),
 ((56, 21), 181),
 ((58, 19), 210),
 ((120, 24), 394),
 ((117, 17), 492),
 ((120, 23), 598),
 ((118, 21), 471),
 ((119, 22), 572),
 ((114, 18), 28),
 ((115, 25), 226),
 ((120, 24), 250),
 ((117, 21), 579),
 ((120, 23), 454),
 ((119, 23), 410),
 ((54, 20), 180),
 ((20, 8), 48),
 ((120, 16), 57),
 ((117, 25), 209),
 ((120, 20), 325),
 ((113, 17), 9),
 ((111, 12), 145),
 ((120, 12), 500),
 ((119, 21), 176),
 ((120, 17), 533),
 ((117, 23), 111),
 ((120, 24), 106),
 ((116, 20), 183),
 ((120, 20), 590),
 ((120, 14), 9),
 ((120, 22), 188),
 ((120, 21), 392),
 ((113, 24), 225),
 ((120, 25), 394),
 ((120, 19), 567),
 ((118, 15), 497),
 ((116, 22), 87),
 ((111, 21), 333),
 ((117, 23), 376),
 ((120, 24), 371),
 ((8, 2), 11),
 ((115, 24), 231),
 ((115, 17), 522),
 ((119, 20), 416),
 ((57, 16), 222),
 ((119, 25), 300),
 ((111, 16), 3),
 ((119, 17), 5

In [28]:
# submit(result, part="a", day=DAY, year=YEAR)

# Part 2

In [21]:
data = copy.deepcopy(dummy)
wall, bliz, start, end, nrows, ncols = data
bliz_states, period = get_blizzard_states(bliz)

first_stop = dfs(set(), (start, 0), end)
second_stop = dfs(set(), first_stop, start)
result = dfs(set(), second_stop, end)[1]
result 


877

In [22]:
# submit(result, part="b", day=DAY, year=YEAR)

That's the right answer!  You are one gold star closer to collecting enough star fruit.You have completed Day 24! You can [Shareon
  Twitter
Mastodon] this victory or [Return to Your Advent Calendar].


<Response [200]>